## Pre-Processing into Spectrograms  

In [ ]:
import numpy as np
import librosa
import os
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

from utils import compute_all_spectrograms

In [ ]:
input_folder = "BallroomData"  # Replace with audio file folder path
output_folder = "spectrograms11"

compute_all_spectrograms(input_folder, output_folder)

## Creating Dataloaders

In [4]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from dataset import BallroomDataset
from model import BeatTrackingNet

In [5]:
spectrogram_dir = 'spectrograms'
annotation_dir = 'BallroomAnnotations'
training_dataset = BallroomDataset(spectrogram_dir, annotation_dir)

indices = list(range(len(training_dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_sub_dataset = torch.utils.data.Subset(training_dataset, train_indices)
test_sub_dataset = torch.utils.data.Subset(training_dataset, test_indices)

print(f'train: {len(train_sub_dataset)}, test: {len(test_sub_dataset)}')

train: 558, test: 140


In [6]:
it = iter(train_sub_dataset)
x, y = next(it)
print(f'x: {x.shape}, target: {y.shape}')

x: torch.Size([3000, 81]), target: torch.Size([3000])


## Creating Models

In [7]:
model_fuz = BeatTrackingNet(input_dim=81, num_filters=16, kernel_size=5, num_layers=11)
model_no_fuz = BeatTrackingNet(input_dim=81, num_filters=16, kernel_size=5, num_layers=11)

a = torch.randn(1, 1, 3000, 81)
print(model_fuz(a).shape)
print(model_no_fuz(a).shape)

torch.Size([1, 3000])
torch.Size([1, 3000])


## Training

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'on {device}')

on cuda


In [9]:
batch_size = 1

train_loader = DataLoader(train_sub_dataset, batch_size=batch_size, shuffle=True)

In [10]:
criterion = torch.nn.BCELoss()

optimizer_fuz = torch.optim.Adam(model_fuz.parameters(), lr=0.001)
optimizer_no_fuz = torch.optim.Adam(model_no_fuz.parameters(), lr=0.001)

In [ ]:
num_epochs = 200

model_fuz = model_fuz.to(device)

for epoch in range(num_epochs):
    model_fuz.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for spectrogram, target in progress_bar:
        spectrogram, target = spectrogram.to(device).unsqueeze(1), target.to(device)

        optimizer_fuz.zero_grad()
        output = model_fuz(spectrogram)

        loss = criterion(output, target)
        loss.backward()
        optimizer_fuz.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


torch.save(model_fuz.state_dict(), 'epoch200_fuz.pt')
print("complete")

In [11]:
training_dataset.fuzziness = False

In [ ]:
num_epochs = 200

model_no_fuz = model_no_fuz.to(device)

for epoch in range(num_epochs):
    model_no_fuz.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for spectrogram, target in progress_bar:
        spectrogram, target = spectrogram.to(device).unsqueeze(1), target.to(device)

        optimizer_no_fuz.zero_grad()
        output = model_no_fuz(spectrogram)

        loss = criterion(output, target)
        loss.backward()
        optimizer_no_fuz.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


torch.save(model_no_fuz.state_dict(), 'epoch200_no_fuz.pt')
print("complete")

Epoch 1/200 - Loss: 0.1326


Epoch 2/200 - Loss: 0.1046


Epoch 3/200 - Loss: 0.1042


Epoch 4/200 - Loss: 0.0872


Epoch 5/200 - Loss: 0.0736


...

Epoch 145/200 - Loss: 0.0482


Epoch 146/200 - Loss: 0.0484


Epoch 147/200 - Loss: 0.0482


Epoch 148/200 - Loss: 0.0485


Epoch 149/200:  45%|████▍     | 249/558 [00:04<00:06, 44.66it/s, loss=0.0869]